
# CAE-LSTM Reduced Order Model——Shu-Osher problem

## Introduction

In order to effectively reduce the design cost and cycle time of using CFD methods, the reduced-order model (ROM) has gained wide attention in recent years. For complex compressible flows, using linear methods such as Proper Orthogonal Decomposition (POD) for flow field dimensionality reduction requires a large number of modes to ensure the reconstruction accuracy. It has been shown that the modes number can be effectively reduced by using nonlinear dimensionality reduction methods. Convolutional Autoencoder (CAE) is a kind of neural network composed of encoder and decoder, which can realize data dimensionality reduction and recon-struction, and can be regarded as a nonlinear extension of POD method. CAE is used for nonlinear dimension-ality reduction, and Long Short-Term Memory (LSTM) is used for time evolution. The CAE-LSTM can obtain high reconstruction and prediction accuracy on the premise of using less latents for unsteady compressible flows.

## Framework of CAE-LSTM

The CAE-LSTM reduced order model uses a CAE network to reduce the dimensionality of the flow field, extract the characteristics of the flow data, compress it into the hidden space of the encoder, and then use the LSTM network to perform coefficient time evolution on the free variables in the hidden space to obtain the free variables at other times of flow. Then, the decoder of the CAE network decodes the evolved free variables and reconstructs the flow field flow data at the corresponding time. The construction of the CAE-LSTM flow reduction model relies on the data reduction of the CAE network and the coefficient time evolution of the LSTM network. Compared with existing methods such as POD/DMD, using CAE networks for nonlinear dimensionality reduction of flow field data and LSTM networks for equation free evolution of free variables can achieve higher compression ratios and improve the efficiency of flow field prediction while ensuring the accuracy of the flow field reduction model.

+ Input：Input the flow field for a period of time.
+ Compression：Extract high-dimensional spatiotemporal flow characteristics by dimensionality reduction of the flow field using the encoder of CAE.
+ Evolution：Learning the evolution of spatiotemporal characteristics of low dimensional spatial flow fields through LSTM and predicting the next moment.
+ Reconstruction：Restore the predicted low-dimensional features of the flow field to high-dimensional space through the decoder of CAE.
+ Output：Output the predicted results of the transient flow field at the next moment.

The first step is to train the CAE network. After the training is completed, the CAE encoder is used to obtain the low dimensional features of the flow field. This low dimensional feature is used as the dataset of the LSTM network for LSTM network training.

![CAE-LSTM.png](./images/CAE-LSTM1.png)

## Training environment

Import the required function library for training, where src includes dataset creation functions, network models, and training loss visualization functions.

The static GRAPH of Mindspore framework is adopted for training. Training can be done on GPU(default) or Ascend(single card).

In [1]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindflow.utils import load_yaml_config

from src import create_cae_dataset, CaeNet, plot_train_loss

In [2]:
np.random.seed(0)
set_seed(0)

In [3]:
parser = argparse.ArgumentParser(description='cae net for shu_osher')
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

## CAE training parameter settings

Import parameter configurations for the dataset, CAE model, and optimizer from the config.yaml file.

In [4]:
config = load_yaml_config(args.config_file_path)
data_params = config["cae_data"]
model_params = config["cae_model"]
optimizer_params = config["cae_optimizer"]

The default path for saving loss files during training is optimizer_params ["summary_dir"], the weight parameters are saved in the ckpt folder.

In [5]:
summary_dir = optimizer_params["summary_dir"]
if not os.path.exists(summary_dir):
    os.mkdir(summary_dir)
ckpt_dir = os.path.join(summary_dir, 'ckpt')
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## Construct CAE neural network

The CAE network consists of six layers of convolution and maximum pooling to form an encoder, and seven layers of convolution and upsampling to form a decoder. Use MSELoss loss function and Adam optimizer.

In [6]:
cae = CaeNet(model_params["data_dimension"], model_params["conv_kernel_size"], model_params["maxpool_kernel_size"],
             model_params["maxpool_stride"], model_params["encoder_channels"], model_params["decoder_channels"])

loss_fn = nn.MSELoss()
cae_opt = nn.Adam(cae.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])

if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(cae, 'O1')
else:
    loss_scaler = None

## CAE dataset

Dataset download address: data_driven/cae-lstm/shu_osher/dataset

The dataset in this case is the numerical simulation results of a one-dimensional shu-osher problem, the Shu-Osher problem is a typical one-dimensional shock wave interaction problem. The coordinate range is \[-5, 5\], and the calculation time t ranges from \[0, 1.8\]. A total of 2093 flow field snapshots, each with a matrix size of 512.

After importing the dataset, perform data sinking settings.

In [7]:
cae_dataset, _ = create_cae_dataset(data_params["data_path"], data_params["batch_size"])

sink_process = data_sink(train_step, cae_dataset, sink_size=1)
train_data_size = cae_dataset.get_dataset_size()

## CAE training

Build forward_fn and train_step, start training the CAE network and visualize the training loss.

In [8]:
def forward_fn(data, label):
    logits = cae(data)
    loss = loss_fn(logits, label)
    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, cae_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, cae_opt(grads))
    return loss

print(f"====================Start cae train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    cae.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(cae, f"{ckpt_dir}/cae_{epoch}.ckpt")
print(f"=====================End cae train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "cae")

pid:4071
====================Start cae train=======================
epoch: 1 train loss: 0.22447173 epoch time: 16.22s
epoch: 2 train loss: 0.07746988 epoch time: 0.69s
epoch: 3 train loss: 0.051422045 epoch time: 0.69s
epoch: 4 train loss: 0.05668847 epoch time: 0.70s
epoch: 5 train loss: 0.042482834 epoch time: 0.70s
...
epoch: 4396 train loss: 0.0013400748 epoch time: 0.69s
epoch: 4397 train loss: 0.001484146 epoch time: 0.69s
epoch: 4398 train loss: 0.0012656535 epoch time: 0.72s
epoch: 4399 train loss: 0.0015285912 epoch time: 0.71s
epoch: 4400 train loss: 0.0011815348 epoch time: 0.74s
====================End cae train=======================


## CAE flow field reconstruction results

After training the CAE network, run cae_prediction.py to view the training results of CAE to determine whether to continue training the LSTM network

The following figures show the real flow field, CAE flow field reconstruction results, and the error curves between them. The first two flow field results show the variation of density at different x positions in the flow field over time, while the third error curve shows the average relative error of the CAE reconstructed flow field and the real flow field label over time. The error remains below 0.02 for most of the time, meeting the accuracy requirements for flow field reconstruction.

<figure class="harf">
    <img src="./images/true.gif" title="cae_train_loss" width="300"/>
    <img src="./images/cae.gif" title="cae_prediction" width="300"/>
    <img src="./images/cae_error.png" title="cae_error" width="300"/>
</center>

## LSTM framework and training Settings

The construction of LSTM network framework, training environment, and other related processing are similar to those of CAE network.

Firstly, import the LSTM network dataset setting parameters, LSTM model, and optimizer parameter settings. The default training loss save path is optimizer_params ["summary_dir"], the weight parameters are saved in the ckpt folder. The network consists of multiple LSTM layers and a full connection layer, using MSELoss loss function and Adam optimizer.

In [9]:
# prepare params
config = load_yaml_config(args.config_file_path)
data_params = config["lstm_data"]
model_params = config["lstm_model"]
optimizer_params = config["lstm_optimizer"]

# prepare summary file
summary_dir = optimizer_params["summary_dir"]
ckpt_dir = os.path.join(summary_dir, 'ckpt')

# prepare model
lstm = Lstm(model_params["latent_size"], model_params["hidden_size"], model_params["num_layers"])

loss_fn = nn.MSELoss()
lstm_opt = nn.Adam(lstm.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])
if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(lstm, 'O1')
else:
    loss_scaler = None

## LSTM dataset loading and processing

The LSTM network dataset is obtained by the CAE encoder, and data sinking settings are performed after creating the dataset.

In [10]:
# prepare dataset
latent_true = cae_prediction()
lstm_dataset, _ = create_lstm_dataset(latent_true, data_params["batch_size"], data_params["time_size"],
                                      data_params["latent_size"], data_params["time_window"],
                                      data_params["gaussian_filter_sigma"])

# data sink
sink_process = data_sink(train_step, lstm_dataset, sink_size=1)
train_data_size = lstm_dataset.get_dataset_size()

## LSTM training

Build forward_fn and train_step, start training the LSTM network and visualize the training loss.

In [11]:
# Define forward function
def forward_fn(data, label):
    logits = lstm(data)
    loss = loss_fn(logits, label)
    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

# Get gradient function
grad_fn = ops.value_and_grad(forward_fn, None, lstm_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, lstm_opt(grads))
    return loss

print(f"====================Start lstm train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    lstm.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(lstm, f"{ckpt_dir}/lstm_{epoch}.ckpt")
print(f"=====================End lstm train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "lstm")

pid:1250
====================Start lstm train=======================
epoch: 1 train loss: 3.9012916 epoch time: 7.68s
epoch: 2 train loss: 5.255037 epoch time: 4.16s
epoch: 3 train loss: 3.9790413 epoch time: 4.25s
epoch: 4 train loss: 4.9073987 epoch time: 4.19s
epoch: 5 train loss: 4.960736 epoch time: 4.20s
...
epoch: 4396 train loss: 0.0003251157 epoch time: 4.15s
epoch: 4397 train loss: 0.0002166091 epoch time: 4.16s
epoch: 4398 train loss: 0.00026683844 epoch time: 4.19s
epoch: 4399 train loss: 0.00017061326 epoch time: 4.21s
epoch: 4400 train loss: 0.00028196868 epoch time: 4.11s
====================End lstm train=======================


## Visualization of predicted flow field results

Run cae_lstm_prediction.py to view the prediction results of the CAE-LSTM reduced order model

The following figures show the actual flow field, the predicted results of the CAE-LSTM network, and the corresponding average relative error. The error of CAE-LSTM prediction results is greater than that of CAE reconstruction because the former has more LSTM evolution error than the latter, but the overall prediction time error remains below 0.04, meeting the accuracy requirements of flow field prediction.

<figure class="harf">
    <img src="./images/true2.gif" title="cae_prediction" width="300"/>
    <img src="./images/cae_lstm.gif" title="cae_lstm_prediction" width="300"/>
    <img src="./images/cae_lstm_error.png" title="cae_lstm_error" width="300"/>
</center>